In [45]:
import numpy as np
import scipy as sp
import os 
import sys
import importlib
import copy
import meal_detection_utils as mdu
import result_utils as resu


In [46]:
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils' if 'C:' in os.getcwd() else './myutils'
sys.path.append(util_path)
import my_file_utils as mfileu
import my_data_process_utils as mdpu

In [47]:
annots = mfileu.read_file('data', 'free_annots_steven_processed.pkl')
bites_baseline_our = mfileu.read_file('bites', 'bites_rf_g3.pkl')
subj_count = len(annots)

In [48]:
def get_results(bites, exclude_subj, max_bite_distance, min_bite_count):
    gts, acovs, clcovs = [], [], []
    
    for subj in range(len(bites)):
        if subj==exclude_subj:
            continue
            
        for sess in range(len(bites[subj])):            
            a = annots[subj][sess]                        
            b = bites[subj][sess]

            episodes = mdu.cluster_bites_by_max_distance(b, max_bite_distance, min_bite_count)
            
            gt, ac, clc = resu.get_meal_detection_results_free(a, clusters=episodes)            
            
            gts = gt if len(gts)==0 else np.concatenate((gts, gt))
            acovs = ac if len(acovs)==0 else np.concatenate((acovs, ac))
            clcovs = clc if len(clcovs)==0 else np.concatenate((clcovs, clc))
            
    res = resu.get_metric_results_free(gts, acovs, clcovs)    
    return res

In [49]:
def get_bites_best_proba(max_bite_distance, min_bite_count):
    best_bites = copy.deepcopy(annots)
    
    for subj in range(subj_count):
        best_proba_th = -1
        best_f1 = -1
    
        for p in range(5, 100, 5):
            proba_th = p/100
            bites = bites_baseline_our[proba_th]
            res = get_results(bites, subj, max_bite_distance, min_bite_count)
            
            if best_proba_th<0 or best_f1<res["f1"]:
                best_proba_th = proba_th
                best_f1 = res["f1"]
        
        #print(subj, best_proba_th, best_f1)
        best_bites[subj] = bites_baseline_our[best_proba_th][subj]
        
    return best_bites


In [50]:
max_res = []
for max_bite_distance  in range(15, 150, 15):
    for min_bite_count in range(2, 8):
            print(max_bite_distance, min_bite_count, end=" | ")
            #for p in range(5, 100, 5):
            #proba_th = p/100
            #bites = bites_baseline_our[proba_th]
            bites = get_bites_best_proba(max_bite_distance=max_bite_distance*16, min_bite_count=min_bite_count)
            res = get_results(bites, exclude_subj=-1, max_bite_distance=max_bite_distance*16, min_bite_count=min_bite_count)

            if len(max_res)== 0 or res["f1"]>max_res["f1"]:
                max_res = res
                max_res['mbp']=min_bite_count
                max_res['mbd']=max_bite_distance

        #print(min_bite_count, max_bite_distance , res["precision"], res["recall"], res["f1"])
    

15 2 | 15 3 | 15 4 | 15 5 | 15 6 | 15 7 | 30 2 | 30 3 | 30 4 | 30 5 | 30 6 | 30 7 | 45 2 | 45 3 | 45 4 | 45 5 | 45 6 | 45 7 | 60 2 | 60 3 | 60 4 | 60 5 | 60 6 | 60 7 | 75 2 | 75 3 | 75 4 | 75 5 | 75 6 | 75 7 | 90 2 | 90 3 | 90 4 | 90 5 | 90 6 | 90 7 | 105 2 | 105 3 | 105 4 | 105 5 | 105 6 | 105 7 | 120 2 | 120 3 | 120 4 | 120 5 | 120 6 | 120 7 | 135 2 | 135 3 | 135 4 | 135 5 | 135 6 | 135 7 | 

In [52]:
for key, val in max_res.items():
    print("{} : {}".format(key, val))

total : 82
total_meal : 49
total_snack : 33
tp : 38
tp_meal : 30
tp_snack : 8
recall : 0.4634146341463415
recall_meal : 0.6122448979591837
recall_snack : 0.24242424242424243
fp : 13
precision : 0.7450980392156863
f1 : 0.5714285714285715
start_error : 108.98094512195122
start_error_meal : 155.9859693877551
start_error_snack : 39.18560606060606
end_error : 143.3559451219512
end_error_meal : 215.03826530612244
end_error_snack : 36.91856060606061
fragment_error : 0.12195121951219512
fragment_error_meal : 0.16326530612244897
fragment_error_snack : 0.06060606060606061
mbp : 4
mbd : 135
